In [ ]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from keras import callbacks
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import pandas as pd
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Dataset paths
train_dir = "/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train"
validation_dir = "/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation" 
test_dir = "/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test"

# Image parameters for consistency
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 2

In [ ]:
# Load datasets using tf.keras.utils.image_dataset_from_directory for better performance
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42  
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=42
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

In [ ]:
# Get class names
class_names = train_ds.class_names
print("Class Names:", class_names)

In [ ]:
# Function to count classes in dataset
def count_classes(dataset, dataset_name):
    class_counts = {class_names[0]: 0, class_names[1]: 0}
    
    for images, labels in dataset:
        for label in labels:
            class_counts[class_names[label]] += 1
    
    print(f"\n{dataset_name} Dataset Distribution:")
    for class_name, count in class_counts.items():
        print(f"{class_name}: {count}")
    
    # Visualize distribution
    plt.figure(figsize=(8, 5))
    plt.bar(class_counts.keys(), class_counts.values(), color=['skyblue', 'lightcoral'])
    plt.title(f'Class Distribution in {dataset_name} Dataset')
    plt.ylabel('Number of Images')
    plt.xlabel('Classes')
    for i, v in enumerate(class_counts.values()):
        plt.text(i, v + 10, str(v), ha='center', va='bottom')
    plt.show()
    
    return class_counts

In [ ]:
# Count and visualize class distributions
train_counts = count_classes(train_ds, "Training")
val_counts = count_classes(validation_ds, "Validation")
test_counts = count_classes(test_ds, "Test")

In [ ]:
# Display sample images from training set
plt.figure(figsize=(15, 10))
for images, labels in train_ds.take(1):
    for i in range(min(16, len(images))):  # Show up to 16 images
        plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"{class_names[labels[i]]}")
        plt.axis("off")
plt.suptitle('Sample Images from Training Dataset', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

# Data augmentation for training set only
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

In [ ]:
# Preprocessing function
def preprocess_data(ds, shuffle=False, augment=False):
    # Resize and rescale
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y), num_parallel_calls=AUTOTUNE)
    
    if shuffle:
        ds = ds.shuffle(1000)
    
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
    
    return ds.prefetch(buffer_size=AUTOTUNE)

# Apply preprocessing
train_ds = preprocess_data(train_ds, shuffle=True, augment=True)
validation_ds = preprocess_data(validation_ds)
test_ds = preprocess_data(test_ds)

In [ ]:
# Create a hybrid model combining MobileNetV2 with custom layers
def create_face_mask_model():
    # Base model - MobileNetV2 for feature extraction
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze base model initially
    base_model.trainable = False
    
    # Add custom classification head
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dense(NUM_CLASSES, activation='softmax')  # Changed to softmax for better probability distribution
    ])
    
    return model, base_model

In [ ]:
# Create the model
model, base_model = create_face_mask_model()

# Display model architecture
model.summary()

In [ ]:
# Compile model with appropriate loss function
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # Since labels are integers (0, 1)
    metrics=['accuracy']
)

In [ ]:
# Define callbacks for better training
callbacks_list = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=0.0001,
        verbose=1
    ),
    ModelCheckpoint(
        'best_face_mask_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Train the model
print("Starting model training...")
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=30,
    callbacks=callbacks_list,
    verbose=1
)

In [ ]:
# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with lower learning rate for fine-tuning
model.compile(
    optimizer=Adam(learning_rate=0.0001/10),  # Lower learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Continue training with fine-tuning
fine_tune_epochs = 10
total_epochs = len(history.history['loss']) + fine_tune_epochs

history_fine = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=total_epochs,
    initial_epoch=len(history.history['loss']),
    callbacks=callbacks_list,
    verbose=1
)

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_ds, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Get predictions for confusion matrix
y_pred_prob = model.predict(test_ds)
y_pred = np.argmax(y_pred_prob, axis=1)

# Get true labels
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Combine training histories
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(15, 5))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.axvline(x=len(history.history['loss']), color='r', linestyle='--', label='Fine-tuning starts')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.axvline(x=len(history.history['loss']), color='r', linestyle='--', label='Fine-tuning starts')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Save the complete model
model.save("face_mask_detection_final.h5")
print("Model saved as 'face_mask_detection_final.h5'")

# Also save in SavedModel format for deployment
model.save("face_mask_detection_savedmodel.keras")
print("Model saved as 'face_mask_detection_savedmodel' directory")

In [ ]:
def predict_mask(image_path, show_image=True):
    """
    Predict if a person is wearing a mask in the given image
    Optimized for OpenCV deployment
    """
    try:
        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        
        # Make prediction
        predictions = model.predict(img_array, verbose=0)
        confidence = np.max(predictions[0])
        predicted_class = np.argmax(predictions[0])
        
        # Display results
        if show_image:
            plt.figure(figsize=(8, 6))
            plt.imshow(img)
            plt.axis('off')
            plt.title(f'Prediction: {class_names[predicted_class]} (Confidence: {confidence:.2%})')
            plt.show()
        
        print(f"Image: {image_path}")
        print(f"Prediction: {class_names[predicted_class]}")
        print(f"Confidence: {confidence:.2%}")
        print(f"Raw predictions: {predictions[0]}")
        print("-" * 50)
        
        return predicted_class, confidence
        
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return None, None

In [ ]:
print("\n" + "="*60)
print("MODEL READY FOR OPENCV DEPLOYMENT")
print("="*60)

print("""
To use this model with OpenCV for real-time face detection:

1. Load the saved model:
   model = tf.keras.models.load_model('face_mask_detection_final.h5')

2. Use with OpenCV face detection:
   face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

3. For each detected face:
   - Resize to (224, 224)
   - Normalize pixel values (divide by 255.0)
   - Use model.predict() to classify

4. Key preprocessing steps for OpenCV integration:
   - Convert BGR to RGB: cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
   - Resize: cv2.resize(face, (224, 224))
   - Normalize: face_array = face_array / 255.0
   - Expand dims: np.expand_dims(face_array, axis=0)

Model Performance Summary:
- Input Shape: (224, 224, 3)
- Classes: {class_names}
- Test Accuracy: {test_accuracy:.2%}
- Model Size: Optimized for real-time inference
""")

print("Training completed successfully!")
print("Files saved:")
print("- face_mask_detection_final.h5 (for loading in Python)")
print("- face_mask_detection_savedmodel/ (for deployment)")
print("- best_face_mask_model.h5 (best checkpoint during training)")